## velocity analysis for myeloid cells

**Author:Dr.Xi Li**

**Created: 07/28/2021**

**Language: Python**



In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import scvelo as scv
import loompy
import pandas as pd
import numpy as np
import os
import scanpy as sc
import scipy.stats as stats
from io import StringIO
from sklearn import linear_model
from sklearn.svm import SVR
import seaborn as sns
import scanpy.external as sce
%matplotlib inline

In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')
# for beautified visualization


plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

## input sample path

First lets find the directory that we are working in

**Note:  SigsDir must be changed depending on the user to allow for proper use of this script**

In [ ]:
os.getcwd()

In [ ]:
#set directory to where signature gene lists locates
SigsDir="/mnt/533ee9c3-18c0-4c72-a09e-d9ce5a10ef9e/sig"
AnnoRefDir="/mnt/533ee9c3-18c0-4c72-a09e-d9ce5a10ef9e/T_anno_ref"

## Unique Gene Names

First we want to select the genes that we wish to look at and format them into a data frame that we can use later on

In [ ]:
#set the directory for outputs
project_ID="pool_M1_M2_MKI67"
scv.settings.figdir=f'{project_ID}_figures'

In [ ]:
adata=scv.read (f"{project_ID}_figures/{project_ID}_anno_dy_reg4.h5ad")

In [ ]:
#store a h5ad file for further faster loadinig
adata.var_names_make_unique("_")

#we need to make a directory to store the files in
try:
    directoryName=project_ID+"_figures"
    os.mkdir(directoryName)
except:
    pass

#we need to make a directory to store the files in
#we need to make a directory to store the files in
try:
    directoryName=project_ID+"_DEG"
    os.mkdir(directoryName)
except:
    pass


In [ ]:
import densmap

In [ ]:
#This section can create a densmap if disired 
embedding, ro, re = densmap.densMAP(n_neighbors=150, n_epochs=500, dens_frac=0.3,
                                    min_dist=0.2,spread=2,
                                    dens_lambda=0.7).fit_transform(adata.to_df(layer="Ms"))
    
#input densmap information into adata
adata.obsm["X_densmap"]=embedding

# TRAJECTORY

In [ ]:
adata = sc.read(f'{project_ID}_figures/{project_ID}_anno.h5ad')

In [ ]:
#calculate velocity with stochastic mode first
scv.tl.velocity(adata,  mode='stochastic',use_latent_time=False)
scv.tl.velocity_graph(adata)

In [ ]:
#calculate and plot out pseudotime
#scv.tl.terminal_states(adata, self_transitions=False)
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color='velocity_pseudotime', cmap='gnuplot', perc=(2,98), save="pseudotime.pdf",basis="umap")

In [ ]:
#we have to calculate velocity first before we recover dynamics 
scv.tl.recover_dynamics(adata ,plot_results=False, 
                        fit_connected_states=True, use_raw=False, n_jobs=20,
                        fit_scaling=True)

In [ ]:
#calculate velocity one more time with dynamical mode
scv.tl.velocity(adata,  mode='dynamical',use_latent_time=True)
scv.tl.velocity_graph(adata)

In [ ]:
#calculate and plot out dynamical pseudotime
#scv.tl.terminal_states(adata, self_transitions=False)
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color='velocity_pseudotime', cmap='gnuplot', perc=(2,98), save="pseudotime.pdf",basis="umap")

In [ ]:
#sometimes we have to do latent-time if the inter population transition is not remarkable, you can refer to their oringinal paper for better understanging of the differences between pseudotime and latent-time
scv.tl.terminal_states(adata, self_transitions=True)
scv.tl.latent_time(adata, vkey='velocity', min_likelihood=0.58, min_confidence=0.9,
                  min_corr_diffusion=0.1, weight_diffusion=True, root_key="root_cells",
                   end_key="end_points", t_max=50, copy=False)
scv.tl.velocity(adata,  mode='dynamical',use_latent_time=True)
scv.tl.velocity_graph(adata)
scv.pl.scatter(adata, color='latent_time', cmap='YlOrRd', perc=(1,99),smooth=5, save="latent_time_tmp.pdf",basis="densmap")

In [ ]:
#summarize the arrows of each cluster into 1 single big arrow
adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

scv.tl.paga(adata,use_time_prior="latent_time", root_key="root_cells", end_key="end_points" #when we only do velocity pseudotime trajectory we set the use_time_prior="velociyt_pseudotime", when we do latent time trajectory we have to chage the prior time into latent time
           ,groups="anno_clusters_sub",
            threshold_root_end_prior=0.95,
           #,groups='velocity_clusters'
           )
df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

In [ ]:
#we can set the color of the big dot into Sample_Cluster so that we will get a pie chart of the percentage of each treatment inside 1 cluster
scv.pl.paga(adata, basis='umap', size=20, alpha=0.7,
            min_edge_width=1.5, node_size_scale=1
            ,save="transition_densmap_anno.svg"
            #,color="velocity_clusters"
            ,color="anno_clusters"
            ,threshold=0.1,minimum_spanning_tree=False,
           )

In [ ]:
#plot velocity and expression plots of each gene in the gene list
scv.pl.velocity(adata,asymm, ncols=2,
                #save="gene_seleted_merge.pdf", 
                basis="umap", color="clusters", figsize=(7,6)
               )

In [ ]:
adata.write(f'{project_ID}_figures/{project_ID}_anno_dy.h5ad')